In [1]:
from ConfigSpace.read_and_write.json import read as cs_read_json
from ConfigSpace import (
    CategoricalHyperparameter, 
    UniformIntegerHyperparameter, 
    UniformFloatHyperparameter, 
    Constant,
)

import pandas as pd
pd.set_option('display.max_colwidth', 999)

In [2]:
with open(
    #'/home/feurerm/projects/2021_posh/60MIN/ASKL_automldata_baseline_full/RF/RF_None_holdout_full_es_nif_75097_0_25_0/space.json'
    '/home/feurerm/projects/2021_posh_new/60MIN/ASKL_automldata_baseline_iter/RF/RF_None_holdout_iterative_es_nif_75097_0_25_0/space.json'
) as fh:
    cs = cs_read_json(fh.read())

In [3]:
cs_as_dict = {}

def format_hp(hp, remove=''):
    if isinstance(remove, str):
        name = hp.name.replace(remove, '')
    else:
        name = hp.name
        for r in remove:
            name = name.replace(r, '')
    name = name.replace(':', ': ').replace('_', ' ').title()
    if isinstance(hp, CategoricalHyperparameter):
        if len(hp.choices) == 1:
            return
        cs_as_dict[name] = {
            'Domain': hp.choices, 
            'Default': hp.default_value,
            'Log': '-',
        }
    elif isinstance(hp, UniformIntegerHyperparameter):
        cs_as_dict[name] = {
            'Domain': '$[%d, %d]$' % (hp.lower, hp.upper), 
            'Default': hp.default_value,
            'Log': 'Yes' if hp.log else 'No',
        }
    elif isinstance(hp, UniformFloatHyperparameter):
        cs_as_dict[name] = {
            'Domain': '$[%s, %s]$' % (str(hp.lower), str(hp.upper)), 
            'Default': hp.default_value,
            'Log': 'Yes' if hp.log else 'No',
        }
    elif isinstance(hp, Constant):
        return
    else:
        raise TypeError(hp)

all_hyperparameters = cs.get_hyperparameters()
all_hyperparameters = sorted(all_hyperparameters, key=lambda t: t.name)
        
for hp in all_hyperparameters:
    if 'classifier' not in hp.name:
        continue
    format_hp(hp, remove='classifier:')
for hp in cs.get_hyperparameters():
    if 'classifier' in hp.name:
        continue
    format_hp(hp, remove=['data_preprocessing:', 'categorical_transformer:', 'numerical_transformer:'])
       
cs_as_pd = pd.DataFrame(cs_as_dict).transpose()
print(cs_as_pd.to_latex(escape=False))
print(cs_as_pd.shape)

\begin{tabular}{llll}
\toprule
{} &                                                                                          Domain &             Default &  Log \\
\midrule
  Choice                                                   &                   (extra_trees, gradient_boosting, mlp, passive_aggressive, random_forest, sgd) &       random_forest &    - \\
Extra Trees: Bootstrap                                     &                                                                                   (True, False) &               False &    - \\
Extra Trees: Criterion                                     &                                                                                 (gini, entropy) &                gini &    - \\
Extra Trees: Max Features                                  &                                                                                    $[0.0, 1.0]$ &                 0.5 &   No \\
Extra Trees: Min Samples Leaf                              &           